# Цель исследования

Необходимо проанализировать базу данных сервиса для чтения книг по подписке

# Загрузка библиотек и создание подключения

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

## Вывод первых 5 строк 

In [3]:
query = '''SELECT * 
           FROM books 
           LIMIT 5
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица содержит данные о книгах:
- book_id — идентификатор книги
- author_id — идентификатор автора
- title — название книги
- num_pages — количество страниц
- publication_date — дата публикации книги
- publisher_id — идентификатор издателя

In [4]:
query = '''SELECT * 
           FROM authors 
           LIMIT 5
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица содержит данные об авторах:
- author_id — идентификатор автора
- author — имя автора

In [5]:
query = '''SELECT * 
           FROM publishers 
           LIMIT 5
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица содержит данные об издательствах:
- publisher_id — идентификатор издательства
- publisher — название издательства

In [6]:
query = '''SELECT * 
           FROM ratings 
           LIMIT 5
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки
- book_id — идентификатор книги
- username — имя пользователя, оставившего оценку
- rating — оценка книги

In [7]:
query = '''SELECT * 
           FROM reviews
           LIMIT 5
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора
- book_id — идентификатор книги
- username — имя автора обзора
- text — текст обзора

## Определим,  сколько книг вышло после 1 января 2000 года

In [8]:
query = '''SELECT COUNT(book_id) 
           FROM books
           WHERE publication_date > '2000-01-01'
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query = '''SELECT b.title AS books,
                  COUNT(DISTINCT r.review_id) AS count_review,
                  AVG(rat.rating) AS avg_rating
           FROM books AS b
           LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
           GROUP BY b.title
           ORDER BY avg_rating DESC, count_review DESC
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,books,count_review,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,Moneyball: The Art of Winning an Unfair Game,3,5.00
2,School's Out—Forever (Maximum Ride #2),3,5.00
3,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,Act of Treason (Mitch Rapp #9),2,5.00
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,3,2.25
995,Drowning Ruth,3,2.00
996,His Excellency: George Washington,2,2.00
997,Junky,2,2.00


Самым высоким показателем является 4 обзора и "5" в качестве средней оценки.

На последнем месте книга Harvesting the Heart с 2 обзорами и средней оценкой в 1.5

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [10]:
query = '''SELECT p.publisher,
                  COUNT(DISTINCT b.title)
           FROM books AS b
           LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY COUNT(DISTINCT b.title) DESC
           LIMIT 1
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


Лидером по выпуску книг толще 50 страниц, является издательство Penguin Books, они выпустили 42 книги

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками

In [11]:
query = '''WITH cut AS (SELECT book_id
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating_id) > 50)
           SELECT a.author,
                  AVG(r.rating)
           FROM books AS b
           INNER JOIN cut ON cut.book_id = b.book_id
           INNER JOIN authors AS a ON b.author_id = a.author_id
           INNER JOIN ratings AS r ON cut.book_id = r.book_id
           GROUP BY author
           ORDER BY AVG(r.rating) DESC
           LIMIT 1
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автором с самой высокой средней оценкой книг, оказалась J.K. Rowling

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''SELECT COUNT(DISTINCT rev.text)/ COUNT(DISTINCT rev.username) AS avg
           FROM ratings AS r
           INNER JOIN reviews AS rev ON r.username = rev.username
           WHERE r.username in (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id) > 48)
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24


В среднем пользователи, поставившие более 48 оценок, пишут 24 обзора